# DESI Forecast: φ-Modulation Analysis

Analysis of φ-modulation detectability with DESI Year 5 data.

This notebook implements a two-parameter extension to ΛCDM testing for log-periodic oscillations in the matter power spectrum at scales determined by the golden ratio φ ≈ 1.618.


In [ ]:
import sys
import os
sys.path.append('../src')

from phi_modulation import PhiModulationModel

import numpy as np
import matplotlib.pyplot as plt

# Set style for publication-quality figures
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10


## Initialize Model

Initialize the PhiModulationModel with default Planck 2018 cosmological parameters.


In [ ]:
# Initialize model
model = PhiModulationModel()

print(f"Golden Ratio φ = {model.phi:.8f}")
print(f"ln(φ) = {model.lnphi:.8f}")
print(f"\nCosmological parameters:")
for key, value in model.params.items():
    print(f"  {key}: {value}")


## Test Different Modulation Amplitudes

Test the sensitivity for different values of A_φ.


In [ ]:
# Test different modulation amplitudes
amplitudes = [0.005, 0.01, 0.02]
results = {}

for A in amplitudes:
    print(f"Computing forecast for A_φ = {A:.3f}...")
    results[A] = model.forecast_desi_sensitivity(A_phi_true=A)


## Generate Forecast Figures

Create a 4-panel figure showing:
1. Power spectrum ratio (P_mod/P_base - 1) vs k
2. BAO signature (ξ(r) comparison)
3. DESI SNR vs amplitude
4. Comparison with existing constraints (placeholder for now)


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Panel 1: Power spectrum ratio
ax = axes[0, 0]
for A in amplitudes:
    ratio = results[A]['Pk_mod'] / results[A]['Pk_base'] - 1
    ax.semilogx(results[A]['k'], ratio, 
                label=f'A_φ = {A:.3f}', linewidth=2)
ax.set_xlabel('k [h/Mpc]')
ax.set_ylabel('ΔP/P')
ax.set_title('φ-modulation in power spectrum')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(0, color='k', linestyle='--', linewidth=0.8, alpha=0.5)

# Panel 2: BAO signature
ax = axes[0, 1]
r, xi_base, xi_mod = model.compute_bao_signature(A_phi=0.01)
ax.plot(r, xi_base, 'k-', label='ΛCDM', alpha=0.7, linewidth=2)
ax.plot(r, xi_mod, 'r--', label='ΛCDM + φ-mod', linewidth=2)
ax.set_xlabel('r [Mpc/h]')
ax.set_ylabel('ξ(r)')
ax.set_title('Correlation function around BAO scale')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 3: DESI detectability
ax = axes[1, 0]
A_values = np.linspace(0.001, 0.02, 20)
snr_values = []

for A in A_values:
    res = model.forecast_desi_sensitivity(A_phi_true=A)
    snr_values.append(res['SNR'])

ax.plot(A_values, snr_values, 'b-', linewidth=2)
ax.axhline(3, color='r', linestyle='--', alpha=0.7, label='3σ detection')
ax.axhline(5, color='g', linestyle='--', alpha=0.7, label='5σ discovery')
ax.set_xlabel('A_φ (modulation amplitude)')
ax.set_ylabel('DESI Y5 SNR')
ax.set_title('DESI Detectability Forecast')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 4: Current constraints (placeholder)
ax = axes[1, 1]
ax.text(0.5, 0.5, 'Comparison with existing constraints\n(Planck, SDSS)\n\nTo be populated with actual data', 
        ha='center', va='center', transform=ax.transAxes, fontsize=11)
ax.set_title('Existing Constraints')
ax.set_xticks([])
ax.set_yticks([])

plt.tight_layout()

# Save figure
os.makedirs('../figures', exist_ok=True)
plt.savefig('../figures/desi_forecasts.png', dpi=300, bbox_inches='tight')
print("Figure saved to ../figures/desi_forecasts.png")
plt.show()


In [ ]:
print("DESI Y5 Forecast Summary:")
print("=" * 40)
for A in amplitudes:
    sigma = results[A]['sigma_Aphi']
    snr = results[A]['SNR']
    print(f"\nA_φ = {A:.3f}:")
    print(f"  σ_A = {sigma:.4f}")
    print(f"  SNR = {snr:.1f}σ")
    print(f"  → {'Detectable' if snr > 3 else 'Marginal'} at {'≥3σ' if snr > 3 else '<3σ'}")
print("\n" + "=" * 40)
print("\nKey conclusion:")
print("DESI Year 5 can detect oscillations with amplitude")
print("A_φ ≳ 0.005 at 3σ confidence.")
